In [1]:
from bbq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
import mplfinance as mpf
from bbq.data.stockdb import StockDB
from bbq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json
import scipy.signal as signal
from sklearn.linear_model import LinearRegression
import numpy as np
from bbq.analyse.tools import *
from bbq.analyse.plot import *
import bbq.log as log

log.setup_logger(level='error')

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()



data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

print('done')


done


In [2]:

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]

ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

wma5 = WMA(data['close'], timeperiod=5)
wma5 = [round(v, 3) for v in wma5]

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('wma5', wma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('close', data['close'], label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')

line.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))

x_index, _, y_extrema = extrema(data, cmp_func=np.greater)

scatter = Scatter()
scatter.add_xaxis(xaxis_data=x_index)
scatter.add_yaxis(
        series_name="极大值",
        y_axis=y_extrema,
        label_opts=opts.LabelOpts(is_show=False), symbol_size=8
    )
line.overlap(scatter)

x_index, _, y_extrema = extrema(data, cmp_func=np.less)
# scatter = Scatter()
# scatter.add_xaxis(xaxis_data=x_index)
# scatter.add_yaxis(
#         series_name="极小值",
#         y_axis=y_extrema,
#         label_opts=opts.LabelOpts(is_show=False),
#     )
scatter = plot_chart(Scatter, x_index, y_extrema, '极小值')
line.overlap(scatter)


a, b, score, x_index, y_index = linear_fitting(data)

line2 = Line()
line2.add_xaxis(list(x_index))
line2.add_yaxis('线性拟合', y_index, label_opts=opts.LabelOpts(is_show=False), is_smooth=True)
line.overlap(line2)

line.render_notebook()


In [5]:
data = data_fund
x = np.array(data['close'])
xmax = signal.argrelextrema(x, np.greater)
ymax = x[xmax[0]]
xmin = signal.argrelextrema(x, np.less)
ymin = x[xmin[0]]

#xmax = np.array(trade_date)[xmax[0]]
#xmin = np.array(trade_date)[xmin[0]]


#转换成numpy的ndarray数据格式，n行1列,LinearRegression需要列格式数据，如下：
X_train = np.array(xmax[0]).reshape((len(xmax[0]), 1))
Y_train = np.array(ymax).reshape((len(ymax), 1))
#新建一个线性回归模型，并把数据放进去对模型进行训练
lineModel = LinearRegression()
lineModel.fit(X_train, Y_train)

#用训练后的模型，进行预测
Y_predict = lineModel.predict(X_train)

#coef_是系数，intercept_是截距，拟合的直线是y=ax+b
a = lineModel.coef_[0][0]
b = lineModel.intercept_[0]
print("y=%.4f*x+%.4f" % (a,b))

#对回归模型进行评分，这里简单使用训练集进行评分，实际很多时候用其他的测试集进行评分
print("得分", lineModel.score(X_train, Y_train))

Y_predict

tt = [x for v in Y_predict for x in v]



y=0.0011*x+1.0983
得分 0.30868520562280444


In [6]:
a, b, score, x_index, y_index0 = linear_fitting(data)
print('a={}, b={}, score={}'.format(a, b, score))



a=0.0013302173747732295, b=1.0762680521004204, score=0.31522599486705605


In [7]:
# codes = await db_fund.load_fund_info(filter={'type': {'$regex': '场内'}}, projection=['code', 'short_name'])
codes = await db_stock.load_stock_info(projection=['code', 'name'])

select = []
for item in codes.to_dict('records'):
#     kdata = await db_fund.load_fund_daily(filter={'code': item['code']}, limit=30, sort=[('trade_date', -1)])
    kdata = await db_stock.load_stock_daily(filter={'code': item['code']}, limit=30, sort=[('trade_date', -1)])
    if kdata is None or kdata.shape[0] < 30:
        continue
    kdata = kdata[::-1]
        
    a, b, score, x_index, y_index = linear_fitting(kdata)
    if a > 1 and score > 0.7:
        print('got you: a={}, score={}, {}({})'.format(a, score, item['name'], item['code']))
        select.append(item)
        
print('done')

got you: a=1.0115889906075377, score=0.8056153007402763, 恒瑞医药(sh600276)
got you: a=1.8470477815699653, score=0.7859297720896538, 片仔癀(sh600436)
got you: a=5.982599539322143, score=0.7690292342354668, 贵州茅台(sh600519)
got you: a=2.2849369819461804, score=0.8947112428173227, 通策医疗(sh600763)
got you: a=3.3449254563187956, score=0.7505141115441846, 山西汾酒(sh600809)
got you: a=1.0185872346310771, score=0.8326991516755069, 隆基股份(sh601012)
got you: a=2.432167160640189, score=0.8396958707172579, 中国中免(sh601888)
got you: a=2.173628012407541, score=0.8760084980873631, 上机数控(sh603185)
got you: a=1.3913762790027377, score=0.9463032538720513, 鸿远电子(sh603267)
got you: a=1.6884258259555172, score=0.8988524933232315, 海天味业(sh603288)
got you: a=1.4412294328053143, score=0.7106657471822998, 万泰生物(sh603392)
got you: a=1.1807682619647357, score=0.9114051384131203, 金域医学(sh603882)
got you: a=1.9395444263481734, score=0.744605877097384, 豪悦护理(sh605009)
got you: a=1.4351231527093593, score=0.7785889978169381, 凯迪股份(sh60528

In [2]:
# data = await db_fund.load_fund_daily(filter={'code': '510150'}, limit=30, sort=[('trade_date', -1)])
data = await db_stock.load_stock_daily(filter={'code': 'sz300999'}, limit=20, sort=[('trade_date', -1)])
data = data[::-1]
kline = my_plot(data)
kline.render_notebook()
